# Phenix - Data preparation
The objective of this notebook is to:
1. [Download Phenix DB tables into csv files](#1.-Download-Phenix-DB-tables-into-csv-files)
9. [Add food group to each Products (Product table)](#2.-Add-food-group-to-each-Products)
9. [Add unit quantity to each Products (Product table)](#3.-Add-unit-quantity-to-each-Products)
9. [Correct gps coordinates for each accounts (Comptes table)](#4.-Correct-gps-coordinates-for-each-Accounts)
9. [Save the corrected tables into csv files](#5.-Save-the-corrected-tables-into-csv-files)

The script requires {phenix_mgt.py} and {off_mgt.py}

In [1]:
import phenix_mgt as phnx
import off_mgt as off
import pandas as pd
import dask
import dask.dataframe as dd

In [2]:
from importlib import reload
phnx = reload(phnx)
off = reload(off)

### 1. Download Phenix DB tables into csv files

In [3]:
# Do this only once to save SQL table into CSV files
# You will need to install Phenix SQL DB (.bak) into your local computer
#phnx.extract_Phenix_SQLtable2CSV(path='./Phenix_DB/')

In [4]:
# List of Phenix tables
lst_table = ['CategorieProduits',
            'CommandeProduits',
            'Commandes',
            'Comptes',
            'OffreProduits',
            'Offres',
            'Produits']

In [5]:
# Import each table one by one
# 'Comptes' table is imported twice for both 'Emetteur' (EC_) and 'Receveur' (RC_)
df_catP = phnx.get_CategorieProduits('./Phenix_DB/' + lst_table[0] + '.csv', prefix='catP_')
df_CP   = phnx.get_CommandeProduits ('./Phenix_DB/' + lst_table[1] + '.csv', prefix='CP_')
df_CO   = phnx.get_Commandes        ('./Phenix_DB/' + lst_table[2] + '.csv', prefix='CO_')
df_EC   = phnx.get_Comptes          ('./Phenix_DB/' + lst_table[3] + '.csv', prefix='EC_')
df_RC   = phnx.get_Comptes          ('./Phenix_DB/' + lst_table[3] + '.csv', prefix='RC_')
df_OP   = phnx.get_OffreProduits    ('./Phenix_DB/' + lst_table[4] + '.csv', prefix='OP_')
df_O    = phnx.get_Offres           ('./Phenix_DB/' + lst_table[5] + '.csv', prefix='O_')
df_P    = phnx.get_Produits         ('./Phenix_DB/' + lst_table[6] + '.csv', prefix='P_')

### 2. Add food group to each Products

In [6]:
df_P = phnx.add_foodgroup(df_P,csv_path='./OFF_DB/mehdi_phenix_foodgroup.csv')
df_P.to_csv('./Phenix_DB_clean/Produits.csv',sep='\t',encoding='utf-8')

-- Adding Product food group column --
--> COMPLETED 


### 3. Add unit quantity to each Products

In [7]:
phnx = reload(phnx)
off = reload(off)
df_P = phnx.add_product_qty(df_P)

-- Adding Product quantity column --
WARNING - This function is time consuming.
STEP 1 - 431022 (not std) 515590 (std)


In [8]:
# STEP 2
df_P = phnx.get_qty_from_PhenixCol(df_P)

STEP 2 - 430808 (not std) 515804 (std)


In [9]:
# STEP 3
df_P = phnx.extrapolate_qty_from_productvalue(df_P,df_OP,df_CP)

----> MERGE STEP ---->
['P_Id', 'P_food_group', 'Qty_val', 'Qty_unit', 'Qty_std', 'Qty_approx', 'Qty_method']
430808
----> MERGE STEP ---->
['P_Id', 'P_food_group', 'Qty_val', 'Qty_unit', 'Qty_std', 'Qty_approx', 'Qty_method', 'OP_Id', 'OP_QuantiteValeur', 'OP_QuantiteUnite']
749397
----> MERGE STEP ---->
['P_Id', 'P_food_group', 'Qty_val', 'Qty_unit', 'Qty_std', 'Qty_approx', 'Qty_method', 'OP_QuantiteValeur', 'OP_QuantiteUnite', 'CP_Id', 'CP_QuantiteValeur', 'CP_QuantiteTotale', 'CP_QuantiteUnite', 'CP_MontantTotal']
745620
STEP 3 - 333223 (not std) 613389 (std)


In [10]:
# STEP 4
OFF_csv = './OFF_DB/fr.openfoodfacts.org.products.csv'
df_P = phnx.get_qty_from_OFF(df_P,OFF_csv=OFF_csv)

STEP 4 - 313187 (not std) 633425 (std)


In [11]:
df_P[df_P.Qty_std==False]

,P_Id,CategorieProduit_Id,Compte_Id,P_EAN,P_Nom,P_PoidsUnitaire,P_food_group,Qty,Qty_val,Qty_unit,Qty_std,Qty_approx,Qty_method
P_Id,,,,,,,,,,,,,
1,1.0,2.0,8.0,3266742120681,CHINOIS ROND CHOCOLAT,NaN,NaN,"{'approx': True, 'val': None, 'unit': None, 's...",NaN,None,False,True,1.0
2,2.0,2.0,8.0,3266742120711,TRESSE PEPITES CHOCOLAT,NaN,NaN,"{'approx': True, 'val': None, 'unit': None, 's...",NaN,None,False,True,4.0
3,3.0,2.0,8.0,3266742120780,TRESSE NATURE,NaN,NaN,"{'approx': True, 'val': None, 'unit': None, 's...",NaN,None,False,True,4.0
5,5.0,2.0,8.0,3266742120858,VIENNOIS.PEPITES CHOCO X4,NaN,NaN,"{'approx': True, 'val': None, 'unit': None, 's...",NaN,None,False,True,1.0
8,8.0,2.0,8.0,3266742122005,KOUGLOFF,NaN,NaN,"{'approx': True, 'val': None, 'unit': None, 's...",NaN,None,False,True,1.0
9,9.0,2.0,8.0,3266742120469,BRIOCHE CHOCO PUR B.+CREME PAT.,NaN,NaN,"{'approx': True, 'val': None, 'unit': None, 's...",NaN,None,False,True,1.0
10,10.0,2.0,8.0,3266742120117,BRIOCHE AUX FRUITS PB +CREME P.,NaN,NaN,"{'approx': True, 'val': None, 'unit': None, 's...",NaN,None,False,True,1.0
11,11.0,2.0,8.0,3461181381246,BAGUETTE 62X64CM EN CARTON,NaN,NaN,"{'approx': True, 'val': 3968.0, 'unit': 'CM', ...",3968.0,CM,False,True,1.0
18,18.0,2.0,8.0,3291810012803,MINI DUO CHOCO/NOISETTE 35GPAF,NaN,NaN,"{'approx': True, 'val': 35.0, 'unit': 'GPAF', ...",35.0,GPAF,False,True,1.0


### 4. Correct gps coordinates for each Accounts

In [12]:
###################################
# Geotag Comptes table
# df_geo_matrix is created once from IGN API and saved for later use
###################################
#df_geo_matrix = phnx.geotag_Compte(df_EC,prefix='EC_',IGN_API='ufwu57bxxg5q2jfp72b3j5di')
#df_geo_matrix.to_csv('./Phenix_DB/df_geo_matrix.csv',sep='\t',encoding='utf-8')

In [13]:
# Manually correct geo coordinates (function manual_geotag())
df_geo_matrix = pd.read_csv('./Phenix_DB/df_geo_matrix.csv',sep='\t',encoding='utf-8',index_col=0)
df_geo_matrix = phnx.manual_geotag(df_geo_matrix)

# Apply correction into df_EC et df_RC
df_EC_corr = phnx.replace_geotag(df_EC,df_geo_matrix,prefix='EC_', threshold=1)
df_RC_corr = phnx.replace_geotag(df_RC,df_geo_matrix,prefix='RC_', threshold=1)

58 coordinates has been updated of 2519
58 coordinates has been updated of 2519


In [14]:
# Basic stats of tables
print('--DATAFRAME SIZES--')
print('Commande : %d' % len(df_CO))
print('Com prod : %d' % len(df_CP))
print('Cat prod : %d' % len(df_catP))
print('Offres   : %d' % len(df_O))
print('Off prod : %d' % len(df_OP))
print('Produits : %d' % len(df_P))
print('E_Compte : %d' % len(df_EC))
print('R_Compte : %d' % len(df_RC))
print('')
print('--Produit ID--')
print(len(df_P))
print('df_P : MAX #%d / MIN #%d' % (df_P.P_Id.max(), df_P.P_Id.min()))
print('df_OP: MAX #%d / MIN #%d' % (df_OP.Produit_Id.max(), df_OP.Produit_Id.min()))

--DATAFRAME SIZES--
Commande : 64945
Com prod : 3676721
Cat prod : 20
Offres   : 65522
Off prod : 3651709
Produits : 946612
E_Compte : 2519
R_Compte : 2519

--Produit ID--
946612
df_P : MAX #1040282 / MIN #1
df_OP: MAX #1040282 / MIN #302


### 5. Save the corrected tables into csv files

In [34]:
df_P.reset_index(drop=True,inplace=True)


In [17]:
phenix_clean_path = './Phenix_DB_clean/'
df_catP.to_csv(phenix_clean_path + lst_table[0] + '.csv',sep='\t',encoding='utf-8')
df_CP.to_csv  (phenix_clean_path + lst_table[1] + '.csv',sep='\t',encoding='utf-8')
df_CO.to_csv  (phenix_clean_path + lst_table[2] + '.csv',sep='\t',encoding='utf-8')
df_EC.to_csv  (phenix_clean_path + 'EC' + lst_table[3] + '.csv',sep='\t',encoding='utf-8')
df_RC.to_csv  (phenix_clean_path + 'RC' + lst_table[3] + '.csv',sep='\t',encoding='utf-8')
df_OP.to_csv  (phenix_clean_path + lst_table[4] + '.csv',sep='\t',encoding='utf-8')
df_O.to_csv   (phenix_clean_path + lst_table[5] + '.csv',sep='\t',encoding='utf-8')
df_P.to_csv   (phenix_clean_path + lst_table[6] + '.csv',sep='\t',encoding='utf-8')
print('Phenix DB Tables are cleaned and ready for analysis...')

Phenix DB Tables are cleaned and ready for analysis...
